In [20]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
# kmodes library: https://github.com/nicodv/kmodes
from kmodes.kprototypes import KPrototypes

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

df = pd.read_csv("./augmented_data3.csv", sep=',',na_values=['NULL', ''])
df = df.dropna(subset = ['Price'])


# for price in Price, remove comma, convert to numeric
# df['Price'] = df['Price'].str.replace(',','')

# df['Price']=df['Price'].astype(float)
df['PRICE_RANK'] = df['Price'].rank(ascending=True)
luxury_weights = []
for index, row in df.iterrows():
    if row['PRICE_RANK'] < len(df.index)/3:
        luxury_weights.append(0)
    elif row['PRICE_RANK'] < 2*len(df.index)/3:
        luxury_weights.append(1)
    else:
        luxury_weights.append(2)
df = df.assign(luxury_weight=pd.Series(luxury_weights).values)

# remove cars that are vans or trucks
# df = df[df['Car type'] != 'VAN']
# df = df[df['Car type'] != 'TRUCK']

# # replace 'SUV' and 'RV' with 'SUV/RV', and 'SMALL' and 'COMPACT' with 'SMALL/COMPACT' in CAR TYPE
# df['Car type'] = df['Car type'].replace(['SUV','RV'], 'SUV/RV')
# df['Car type'] = df['Car type'].replace(['SMALL','COMPACT'], 'SMALL/COMPACT')



df.head(10)

# old_names = df.columns.values
# new_names = [name.replace('-','_') for name in old_names]
# df.rename(columns = dict(zip(old_names, new_names)), inplace = True)
# print(df.columns.values)
# print(df2.columns.values)

,Manufacturer,Model,Age,Gender,Region,Sum,EV/Hybrid available,advertisement list API,(Ryvuss Query),link to list page of Encar.com,Unnamed: 10,Car type,Price,Country,PRICE_RANK,luxury_weight
0,Audi,A5,35~39,M,Seoul,747,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,MIDDLE,3605.0,Japan,76.0,1
1,Audi,A7,35~39,M,Seoul,958,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,BIG,5069.0,Japan,94.0,2
2,Audi,NEW A3,35~39,M,Gyeonggi-do,403,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SMALL/COMPACT,1900.0,Japan,34.0,0
3,Audi,New A4,35~39,M,Gyeonggi-do,1144,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SMALL/COMPACT,2416.0,Japan,55.0,1
4,Audi,NEW A6,35~39,M,Gyeonggi-do,2253,O,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,MIDDLE,3303.0,Japan,70.0,1
5,Audi,NEW A8,40~44,M,Gyeonggi-do,617,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,BIG,8016.0,Japan,107.0,2
6,Audi,Q5,35~39,M,Gyeonggi-do,564,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SUV/RV,4117.0,Japan,84.0,2
10,BMW,3-Series (F30),35~39,M,Gyeonggi-do,2318,O,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SMALL/COMPACT,3296.0,Japan,69.0,1
11,BMW,3-Series Gran Turismo (F34),35~39,M,Seoul,687,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SMALL/COMPACT,2950.0,Japan,64.0,1
12,BMW,4-Series (F32),30~34,M,Seoul,1124,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SPORT,2929.0,Japan,63.0,1


In [70]:
# lst = []
# for index, row in df.iterrows():
#     lst.append((row['MNFCNMEN'], row['MDLNMEN'],row['CAR TYPE'],row['MAX PRICE OF 2014']))
# # print(lst)
# # for index2, row2 in df2.iterrows():
# #     for index, row in df.iterrows():
# #         if row2['Manufacturer'] == row['MNFCNMEN'] and row2['Model'] == row['MNFCNM']:
# #         df2.iloc[index2, df2.columns.get_loc('Car type')] = 'hello'
# #             df2.at[index2, 'Car type'] = df.at[index,'CAR TYPE']
# #             df2[index2]['Car type'] = df[index]['CAR TYPE']

# #             row2['Car type'] = row['CAR TYPE']
# #             row2['Price'] = row['MAX PRICE OF 2014']
# # df2.head()
            

In [72]:
# df2['Car type'] = ''
# df2['Price'] = ''

# for index, row in df2.iterrows():
#     for item in lst:
#         if row['Manufacturer'] == item[0] and row['Model'] == item[1]:
#             df2.iloc[index, df2.columns.get_loc('Car type')] = item[2]
#             df2.iloc[index, df2.columns.get_loc('Price')] = item[3]
#             break
# df2.head(100)


,Manufacturer,Model,Age,Gender,Region,Sum,EV/Hybrid available,advertisement list API,(Ryvuss Query),link to list page of Encar.com,Unnamed: 10,Car type,Price
0,Audi,A5,35~39,M,Seoul,747,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,MIDDLE,3605
1,Audi,A7,35~39,M,Seoul,958,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,BIG,5069
2,Audi,NEW A3,35~39,M,Gyeonggi-do,403,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SMALL/COMPACT,1900
3,Audi,New A4,35~39,M,Gyeonggi-do,1144,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SMALL/COMPACT,2416
4,Audi,NEW A6,35~39,M,Gyeonggi-do,2253,O,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,MIDDLE,3303
5,Audi,NEW A8,40~44,M,Gyeonggi-do,617,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,BIG,8016
6,Audi,Q5,35~39,M,Gyeonggi-do,564,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SUV/RV,4117
7,BMW,1-Series (F20),30~34,M,Gyeonggi-do,707,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,,
8,BMW,3-Series,40~44,M,Gyeonggi-do,488,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,,
9,BMW,3-Series (E90),35~39,M,Gyeonggi-do,1131,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,,


In [73]:
# df2.to_csv('augmented_data2.csv', index=False)

In [35]:
# df2.iloc[1, df2.columns.get_loc('Car type')] = 'Hello'
# # df2.head()


,Manufacturer,Model,Age,Gender,Region,Sum,EV/Hybrid available,advertisement list API,(Ryvuss Query),link to list page of Encar.com,Unnamed: 10,Car type,Price
0,Audi,Hello,35~39,M,Seoul,747,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,,
1,Audi,A7,35~39,M,Seoul,958,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,Hello,
2,Audi,NEW A3,35~39,M,Gyeonggi-do,403,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,,
3,Audi,New A4,35~39,M,Gyeonggi-do,1144,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,,
4,Audi,NEW A6,35~39,M,Gyeonggi-do,2253,O,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,,


In [2]:
# df['CAR TYPE'].unique()
# df.dtypes
# df.columns.values

In [29]:
# demographic data
demo = pd.read_csv("./demo_data.csv", sep=',',na_values=['NULL', ''])
print(demo.columns.values)
demo.head(10)


['Manufacturer' 'Model' 'Age' 'Gender' 'Region' 'Week 14' 'Week 15'
 'Week 16' 'Week 17' 'Week 18' 'Week 19' 'Week 20' 'Week 21' 'Week 22'
 'Week 23' 'Week 24' 'Week 25' 'Week 26' 'Week 27' 'Week 28' 'Week 29'
 'Week 30' 'Week 31' 'Week 32' 'Week 33' 'Week 34' 'Week 35' 'Week 36'
 'Week 37' 'Week 38' 'Week 39']


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Manufacturer,Model,Age,Gender,Region,Week 14,Week 15,Week 16,Week 17,Week 18,Week 19,Week 20,Week 21,Week 22,Week 23,Week 24,Week 25,Week 26,Week 27,Week 28,Week 29,Week 30,Week 31,Week 32,Week 33,Week 34,Week 35,Week 36,Week 37,Week 38,Week 39
0,BMW,5-Series (F10),35~39,M,Gyeonggi-do,130,128,125,135,138,151,121,138,147,124,121,108,121,119,119,106,136,112,124,120,113,123,121,123,111,114
1,BMW,5-Series (F10),35~39,M,Seoul,128,117,119,106,122,126,127,120,119,113,115,114,119,140,97,105,107,111,116,123,114,109,105,105,101,98
2,Hyundai,Grandeur HG,35~39,M,Gyeonggi-do,118,122,104,114,115,115,118,112,123,109,111,104,107,98,102,109,121,97,103,104,106,95,114,88,106,92
3,BMW,5-Series (F10),40~44,M,Gyeonggi-do,99,102,92,83,99,96,91,81,82,90,100,88,84,81,85,99,85,86,85,119,92,81,88,89,82,87
4,Mercedes-Benz,E-Class W212,35~39,M,Seoul,96,98,93,100,109,117,118,93,106,101,101,90,95,119,108,95,92,116,104,107,110,95,91,94,90,78
5,Audi,NEW A6,35~39,M,Gyeonggi-do,95,82,85,90,96,71,89,88,94,85,92,73,81,83,82,81,92,77,86,96,108,91,87,92,87,70
6,Mercedes-Benz,E-Class W212,35~39,M,Gyeonggi-do,94,92,102,84,110,120,112,104,97,118,91,101,85,83,108,103,102,105,109,116,118,99,93,95,93,84
7,BMW,5-Series (F10),40~44,M,Seoul,91,79,86,74,93,95,78,86,91,90,88,86,73,72,83,89,84,81,71,94,76,69,74,83,74,74
8,BMW,5-Series (F10),30~34,M,Gyeonggi-do,88,80,79,73,66,85,81,84,100,81,66,77,72,70,78,90,73,84,95,85,59,52,56,68,58,65
9,BMW,3-Series (F30),35~39,M,Gyeonggi-do,86,104,96,99,95,110,88,93,92,87,83,77,83,88,82,74,97,85,79,97,93,82,82,100,94,72


In [30]:
age_approx = []
for index, row in demo.iterrows():
    if row['Age'][0]=='~':
        age_approx.append('0')
    else:
        age_approx.append(row['Age'][0]+'0')
    
demo = demo.assign(age_approx=pd.Series(age_approx).values)
demo = demo.drop(demo[demo.age_approx == 'A0'].index)
demo.head()

,Manufacturer,Model,Age,Gender,Region,Week 14,Week 15,Week 16,Week 17,Week 18,Week 19,Week 20,Week 21,Week 22,Week 23,Week 24,Week 25,Week 26,Week 27,Week 28,Week 29,Week 30,Week 31,Week 32,Week 33,Week 34,Week 35,Week 36,Week 37,Week 38,Week 39,age_approx
0,BMW,5-Series (F10),35~39,M,Gyeonggi-do,130,128,125,135,138,151,121,138,147,124,121,108,121,119,119,106,136,112,124,120,113,123,121,123,111,114,30
1,BMW,5-Series (F10),35~39,M,Seoul,128,117,119,106,122,126,127,120,119,113,115,114,119,140,97,105,107,111,116,123,114,109,105,105,101,98,30
2,Hyundai,Grandeur HG,35~39,M,Gyeonggi-do,118,122,104,114,115,115,118,112,123,109,111,104,107,98,102,109,121,97,103,104,106,95,114,88,106,92,30
3,BMW,5-Series (F10),40~44,M,Gyeonggi-do,99,102,92,83,99,96,91,81,82,90,100,88,84,81,85,99,85,86,85,119,92,81,88,89,82,87,40
4,Mercedes-Benz,E-Class W212,35~39,M,Seoul,96,98,93,100,109,117,118,93,106,101,101,90,95,119,108,95,92,116,104,107,110,95,91,94,90,78,30


In [23]:
name_list = demo.columns.values
name_list

array(['Manufacturer', 'Model', 'Age', 'Gender', 'Region', 'Week 14',
       'Week 15', 'Week 16', 'Week 17', 'Week 18', 'Week 19', 'Week 20',
       'Week 21', 'Week 22', 'Week 23', 'Week 24', 'Week 25', 'Week 26',
       'Week 27', 'Week 28', 'Week 29', 'Week 30', 'Week 31', 'Week 32',
       'Week 33', 'Week 34', 'Week 35', 'Week 36', 'Week 37', 'Week 38',
       'Week 39', 'age_approx'], dtype=object)

In [31]:
#### TEST ####
grouped_demo = demo.groupby(['age_approx','Gender'], as_index=True)
#### END TEST ####

In [43]:
#### TEST ####
df = pd.DataFrame({'Date': ['2015-05-08', '2015-05-07', '2015-05-06', '2015-05-05', '2015-05-08', '2015-05-07', '2015-05-06', '2015-05-05'], 'Sym': ['aapl', 'aapl', 'aapl', 'aapl', 'aaww', 'aaww', 'aaww', 'aaww'], 'Data2': [11, 8, 10, 15, 110, 60, 100, 40],'Data3': [5, 8, 6, 1, 50, 100, 60, 120]})


df.head()

,Date,Sym,Data2,Data3
0,2015-05-08,aapl,11,5
1,2015-05-07,aapl,8,8
2,2015-05-06,aapl,10,6
3,2015-05-05,aapl,15,1
4,2015-05-08,aaww,110,50


In [44]:
#### TEST ####
group = df['Data3'].groupby(df['Date']).sum()

df['Data4'] = group
df.head()

,Date,Sym,Data2,Data3,Data4
0,2015-05-08,aapl,11,5,NaN
1,2015-05-07,aapl,8,8,NaN
2,2015-05-06,aapl,10,6,NaN
3,2015-05-05,aapl,15,1,NaN
4,2015-05-08,aaww,110,50,NaN


In [41]:
#### TEST ####
demo.get_group(('30','M'))

,Manufacturer,Model,Age,Gender,Region,Week 14,Week 15,Week 16,Week 17,Week 18,Week 19,Week 20,Week 21,Week 22,Week 23,Week 24,Week 25,Week 26,Week 27,Week 28,Week 29,Week 30,Week 31,Week 32,Week 33,Week 34,Week 35,Week 36,Week 37,Week 38,Week 39,age_approx
0,BMW,5-Series (F10),35~39,M,Gyeonggi-do,130,128,125,135,138,151,121,138,147,124,121,108,121,119,119,106,136,112,124,120,113,123,121,123,111,114,30
1,BMW,5-Series (F10),35~39,M,Seoul,128,117,119,106,122,126,127,120,119,113,115,114,119,140,97,105,107,111,116,123,114,109,105,105,101,98,30
2,Hyundai,Grandeur HG,35~39,M,Gyeonggi-do,118,122,104,114,115,115,118,112,123,109,111,104,107,98,102,109,121,97,103,104,106,95,114,88,106,92,30
4,Mercedes-Benz,E-Class W212,35~39,M,Seoul,96,98,93,100,109,117,118,93,106,101,101,90,95,119,108,95,92,116,104,107,110,95,91,94,90,78,30
5,Audi,NEW A6,35~39,M,Gyeonggi-do,95,82,85,90,96,71,89,88,94,85,92,73,81,83,82,81,92,77,86,96,108,91,87,92,87,70,30
6,Mercedes-Benz,E-Class W212,35~39,M,Gyeonggi-do,94,92,102,84,110,120,112,104,97,118,91,101,85,83,108,103,102,105,109,116,118,99,93,95,93,84,30
8,BMW,5-Series (F10),30~34,M,Gyeonggi-do,88,80,79,73,66,85,81,84,100,81,66,77,72,70,78,90,73,84,95,85,59,52,56,68,58,65,30
9,BMW,3-Series (F30),35~39,M,Gyeonggi-do,86,104,96,99,95,110,88,93,92,87,83,77,83,88,82,74,97,85,79,97,93,82,82,100,94,72,30
10,BMW,3-Series (F30),30~34,M,Gyeonggi-do,84,95,86,81,96,78,84,90,88,96,91,94,67,58,81,83,75,84,100,107,90,78,93,82,83,69,30
11,Mercedes-Benz,C-Class W205,35~39,M,Seoul,83,92,74,81,82,96,100,83,76,82,83,83,86,96,90,66,83,89,78,73,74,70,64,74,72,65,30


In [19]:
#### TEST ####
# name_list = demo.columns.values
# demo['avg'] = demo[name_list[5:-1]].mean(axis=1)
# demo.head(10)
demo['avg'] = demo.apply

pandas.core.groupby.groupby.DataFrameGroupBy

In [56]:
demo['age_approx'].unique()

array(['30', '40', '20', '50', '60', '70', '10', '0'], dtype=object)

In [57]:
demo.dtypes
name_list = demo.columns.values
for name in name_list[5:]:
    demo[name] = pd.to_numeric(demo[name])
# demo.dtypes  

In [58]:
unique_age = list(demo['age_approx'].unique())
unique_gender = list(demo['Gender'].unique())
table = {}
for age in unique_age:
    for gender in unique_gender:
        table[(age, gender)] = {}
for index, row in demo.iterrows():
    num_list = []
    for name in name_list[5:-1]:
        num_list.append(row[name])
    if row['Model'] in table[(row['age_approx'], row['Gender'])]:
        table[(row['age_approx'], row['Gender'])][row['Model']] += num_list
    else:
        table[(row['age_approx'], row['Gender'])][row['Model']] = num_list
for age_gender, value in table.items():
    for model, num_list in value.items():
        table[age_gender][model] = np.mean(table[age_gender][model])


In [59]:
table

{(30, 'M'): {'5-Series (F10)': 26.76923076923077,
  'Grandeur HG': 27.967548076923077,
  'E-Class W212': 22.358173076923077,
  'NEW A6': 17.873626373626372,
  '3-Series (F30)': 21.53315649867374,
  'C-Class W205': 19.516381766381766,
  'All New Canival': 20.432692307692307,
  'All New Sorento': 17.007051282051282,
  'LF Sonata': 15.329326923076923,
  'Grandeur IG': 15.580645161290322,
  'AVANTE AD': 15.10207100591716,
  '5 Series (G30)': 15.181697612732096,
  'E-Class W213': 17.27171215880893,
  'K5': 13.398110661268555,
  'Genesis DH': 17.800248138957816,
  'Carnival R': 10.346153846153847,
  '3-Series (E90)': 10.410769230769231,
  'Golf MK7': 12.29185520361991,
  'Discovery 4': 9.245421245421245,
  'GENESIS': 12.472257250945775,
  'Santa Fe DM': 13.004273504273504,
  'All new K7': 13.338942307692308,
  'C-Class W204': 10.442684766214178,
  'Avante MD': 11.664835164835164,
  'YF Sonata': 12.914529914529915,
  'Mohave': 10.857142857142858,
  'Malibu': 10.369505494505495,
  'The New K7'

In [60]:
for age_gender, value in table.items():
    new_dict = value.copy()
    for model, tot in table[age_gender].items():
#         print(model in df['Model'])
        if not model in list(df['Model']):
            new_dict.pop(model, None)
    table[age_gender] = new_dict
#     table[age_gender] = list(table[age_gender].items())
table

{(30, 'M'): {'Grandeur HG': 27.967548076923077,
  'E-Class W212': 22.358173076923077,
  'NEW A6': 17.873626373626372,
  '3-Series (F30)': 21.53315649867374,
  'C-Class W205': 19.516381766381766,
  'All New Canival': 20.432692307692307,
  'All New Sorento': 17.007051282051282,
  'LF Sonata': 15.329326923076923,
  '5 Series (G30)': 15.181697612732096,
  'Genesis DH': 17.800248138957816,
  'Carnival R': 10.346153846153847,
  'Golf MK7': 12.29185520361991,
  'Discovery 4': 9.245421245421245,
  'Santa Fe DM': 13.004273504273504,
  'C-Class W204': 10.442684766214178,
  'Mohave': 10.857142857142858,
  'Malibu': 10.369505494505495,
  'The New K7': 12.758620689655173,
  'New A4': 10.997150997150998,
  'CLS-Class W218': 13.5051775147929,
  '4-Series (F32)': 12.819230769230769,
  'RAY': 10.331309041835357,
  'Chevrolet Spark': 8.506072874493928,
  'Golf MK6': 6.947692307692308,
  'New Sorento R': 9.336032388663968,
  '5-Series GT (F07)': 10.003205128205128,
  'all new Morning': 8.96474358974359,


In [61]:
table[(20,'F')]

{'RAY': 1.028205128205128,
 'all new Morning': 1.3461538461538463,
 'New A4': 0.5528846153846154,
 'LF Sonata': 1.7692307692307692,
 'Chevrolet Spark': 0.9846153846153847,
 '3-Series (F30)': 2.2435897435897436,
 'New SM5 Platinum': 0.3384615384615385,
 'Grandeur HG': 1.4487179487179487,
 'C-Class W205': 0.6258741258741258,
 'NEW A6': 0.7564102564102564,
 'A7': 0.5769230769230769,
 'E-Class W212': 1.2115384615384615,
 'Range Rover Evoque': 0.2408906882591093,
 'Range Rover 4th': 0.1794871794871795,
 'Chevrolet Orlando': 0.3131868131868132,
 'Sonata the Brilliant': 0.7692307692307693,
 'Golf MK7': 0.5512820512820513,
 'NEW CC': 0.6153846153846154,
 'New Tiguan': 0.35,
 '3-Series Gran Turismo (F34)': 0.07692307692307693,
 'X6 (F16)': 0.6538461538461539,
 '7-Series (F01)': 0.3230769230769231,
 'Macan': 1.0769230769230769,
 'Cooper': 0.34841628959276016,
 'A5': 0.8974358974358975,
 'The New Avante': 0.25384615384615383,
 'Q50': 0.2692307692307692,
 'Chevrolet Cruze': 0.8269230769230769,
 'C

In [62]:
for age_gender, value in table.items():
    table[age_gender] = list(table[age_gender].items())

In [63]:
table

{(30, 'M'): [('Grandeur HG', 27.967548076923077),
  ('E-Class W212', 22.358173076923077),
  ('NEW A6', 17.873626373626372),
  ('3-Series (F30)', 21.53315649867374),
  ('C-Class W205', 19.516381766381766),
  ('All New Canival', 20.432692307692307),
  ('All New Sorento', 17.007051282051282),
  ('LF Sonata', 15.329326923076923),
  ('5 Series (G30)', 15.181697612732096),
  ('Genesis DH', 17.800248138957816),
  ('Carnival R', 10.346153846153847),
  ('Golf MK7', 12.29185520361991),
  ('Discovery 4', 9.245421245421245),
  ('Santa Fe DM', 13.004273504273504),
  ('C-Class W204', 10.442684766214178),
  ('Mohave', 10.857142857142858),
  ('Malibu', 10.369505494505495),
  ('The New K7', 12.758620689655173),
  ('New A4', 10.997150997150998),
  ('CLS-Class W218', 13.5051775147929),
  ('4-Series (F32)', 12.819230769230769),
  ('RAY', 10.331309041835357),
  ('Chevrolet Spark', 8.506072874493928),
  ('Golf MK6', 6.947692307692308),
  ('New Sorento R', 9.336032388663968),
  ('5-Series GT (F07)', 10.00320

In [64]:
from operator import itemgetter
for age_gender, value in table.items():
    sorted_scores = sorted(value, key=itemgetter(1))
    new_dict = {}
    for i in range(len(sorted_scores)):
        if i <= len(sorted_scores)//3:
            new_dict[sorted_scores[i][0]] = 0
#             new_list.append((sorted_scores[i][0],0))
        elif i <= 2*len(sorted_scores)//3:
            new_dict[sorted_scores[i][0]] = 1
#             new_list.append((sorted_scores[i][0],1))
        else:
            new_dict[sorted_scores[i][0]] = 2
#             new_list.append((sorted_scores[i][0],2))
    table[age_gender] = new_dict
table[(30,'M')]

{'S60': 0,
 'Altima': 0,
 'New XF': 0,
 'Cooper': 0,
 'NX300h': 0,
 'Macan': 0,
 'Ghibli': 0,
 'New SM3': 0,
 'New Touareg': 0,
 'NEW A3': 0,
 'Korando Turismo': 0,
 'Panamera': 0,
 'Maxcruz': 0,
 'All New Pride': 0,
 'M-class W166': 0,
 'Boxster': 0,
 'New Chairman W': 0,
 'GLA-Class X156': 0,
 'Captiva': 0,
 'IS250': 0,
 'Trax': 0,
 'X6 (E71)': 0,
 'G-Class W463': 0,
 'The New Passat': 0,
 'Alpheon': 0,
 'New Camry': 0,
 'Cooper S': 0,
 'All New Carens': 0,
 'Range Rover 4th': 0,
 'Rexton W': 0,
 'New IS250': 0,
 '911': 0,
 'Korando Sports': 0,
 'Range Rover Sport 2nd': 0,
 'QM3': 0,
 'Mustang': 0,
 'NEW A8': 0,
 'New KORANDO C': 0,
 'CTS 3rd': 0,
 'Accent(New)': 1,
 'GLK-Class X204': 1,
 'All New SM7': 1,
 'M3 (F80)': 1,
 'Range Rover Evoque': 1,
 'i40': 1,
 'Aslan': 1,
 'New SLK-Class': 1,
 'Q5': 1,
 'M3 (E90)': 1,
 'Grand Cherokee': 1,
 'The New K9': 1,
 'Q50': 1,
 'K3': 1,
 'The New GENESIS Coupe': 1,
 'SM7 Nova': 1,
 'Golf MK6': 1,
 'K9': 1,
 'Grandeur HG Hybrid': 1,
 'Chevrolet

In [65]:
# table[20]

In [66]:
table

{(30, 'M'): {'S60': 0,
  'Altima': 0,
  'New XF': 0,
  'Cooper': 0,
  'NX300h': 0,
  'Macan': 0,
  'Ghibli': 0,
  'New SM3': 0,
  'New Touareg': 0,
  'NEW A3': 0,
  'Korando Turismo': 0,
  'Panamera': 0,
  'Maxcruz': 0,
  'All New Pride': 0,
  'M-class W166': 0,
  'Boxster': 0,
  'New Chairman W': 0,
  'GLA-Class X156': 0,
  'Captiva': 0,
  'IS250': 0,
  'Trax': 0,
  'X6 (E71)': 0,
  'G-Class W463': 0,
  'The New Passat': 0,
  'Alpheon': 0,
  'New Camry': 0,
  'Cooper S': 0,
  'All New Carens': 0,
  'Range Rover 4th': 0,
  'Rexton W': 0,
  'New IS250': 0,
  '911': 0,
  'Korando Sports': 0,
  'Range Rover Sport 2nd': 0,
  'QM3': 0,
  'Mustang': 0,
  'NEW A8': 0,
  'New KORANDO C': 0,
  'CTS 3rd': 0,
  'Accent(New)': 1,
  'GLK-Class X204': 1,
  'All New SM7': 1,
  'M3 (F80)': 1,
  'Range Rover Evoque': 1,
  'i40': 1,
  'Aslan': 1,
  'New SLK-Class': 1,
  'Q5': 1,
  'M3 (E90)': 1,
  'Grand Cherokee': 1,
  'The New K9': 1,
  'Q50': 1,
  'K3': 1,
  'The New GENESIS Coupe': 1,
  'SM7 Nova': 

In [208]:
# table[50]

{'M3 (E90)': 0,
 'Caristar 2900': 0,
 'Magnum': 0,
 'Viper': 0,
 '612': 0,
 'Elise': 0,
 '200 1st': 0,
 'MR-S': 0,
 'Aveo Sedan': 0,
 '206RC': 0,
 '308SW': 0,
 'DS5': 0,
 'New Concord': 0,
 'Dawn': 0,
 'MAZDA 5': 0,
 'Five Hundred': 0,
 'Elan': 0,
 'Mondeo 3rd': 0,
 'IS350': 0,
 'SRX': 0,
 '히미코': 0,
 'DTS': 0,
 'Compass': 0,
 'Lemans': 0,
 'Town Car': 0,
 'LS': 0,
 'A-Class W169': 0,
 'Potentia': 0,
 'Carens': 0,
 'New Potentia': 0,
 'The new K3 EURO': 0,
 'The new K3 KOUP': 0,
 'Riviera': 0,
 'Vehicross': 0,
 'Yaris(Vitz)': 0,
 'Rover Mini': 0,
 'Titan': 0,
 'Elgrand': 0,
 'Cherokee(KL)': 0,
 'Lanos': 0,
 'XJ-8': 0,
 'Daimler': 0,
 'Challenger': 0,
 'Vandura': 0,
 'Montero': 0,
 'Mulsanne': 0,
 '4C': 0,
 'S70': 0,
 'ENZO FERRARI': 0,
 'xB': 0,
 '2-Series (F22)': 0,
 '8-Series': 0,
 '62s': 0,
 '9000': 0,
 'Q45': 0,
 'Intrepid': 0,
 'Durango': 0,
 'TL': 0,
 'CL': 0,
 '960': 0,
 'MarkX': 0,
 'EB110': 0,
 'Giulietta': 0,
 'Rocksta R2': 0,
 'Murcielago': 0,
 'Blazer': 0,
 'Q3': 0,
 'Tunlan

In [195]:
# table

{30: {'Riviera': 0,
  'CL': 0,
  '280ZX': 0,
  'Gaia': 0,
  'Teana': 0,
  '206SW': 0,
  'Noah': 0,
  'Crown Majesta': 0,
  'Tigra': 0,
  'BLS': 0,
  'Credos 2': 0,
  'Credos': 0,
  'LeSable': 0,
  'BRZ': 0,
  'Moco': 0,
  'Bluebird': 0,
  'Quest': 0,
  'XK8': 0,
  'Mulsanne': 0,
  'Veyron 16.4': 0,
  'MGTF': 0,
  'Soarer': 0,
  'F12 Berlinetta': 0,
  'Alphard': 0,
  'Altezza': 0,
  'Solati': 0,
  '62s Landaulet': 0,
  'Lancia': 0,
  'Viper': 0,
  'Megane': 0,
  'C3': 0,
  'Azure': 0,
  'Cedric': 0,
  'Leaf': 0,
  'Diablo': 0,
  'DB11': 0,
  'Lanos II': 0,
  'FTO': 0,
  'LHS': 0,
  'Wish': 0,
  'Sequoia': 0,
  'Crown': 0,
  'F355': 0,
  'LC': 0,
  'S-MAX': 0,
  'Vanquish S': 0,
  'Reventon': 0,
  'MC12': 0,
  'Pao': 0,
  'Pathfinder': 0,
  'Bentayga': 0,
  'STS': 0,
  'H1': 0,
  'i': 0,
  'iQ': 0,
  'Vehicross': 0,
  'Brera': 0,
  'Elgrand': 0,
  'CJ': 0,
  'Lanos': 0,
  'Vellfire': 0,
  'Explorer Spot Trac': 0,
  'Giulietta': 0,
  'S70': 0,
  'Sentra': 0,
  'FF': 0,
  'Titan': 0,
  'Ma

In [68]:
def second_stage(df, age, gender, brand):
    """
    Recommends a list of cars that satisfies the specified age and preferred brand.
    If brand is a nationality ('Korea', 'Japan', or 'Germany'), add weights to the 
    corresponding car of the associated country. If brand is 'Luxury', add weights 
    to each car according to its price.

    Inputs:
    - df: data frame, data to filter
    - age: string, age in approximation (e.g.,'30' for 30s)
    - gender: stirng, either 'M' or 'F'
    - brand: string, either 'Korea', 'Japan', 'Germany', or 'Luxury'

    Outputs:
    - filtered: dataframe, containing objects satisfying the conditions
    """
    filtered = None
#     ranks = []
    scores = []
    for index, row in df.iterrows():
        score = 0
        if row['Model'] in table[(int(age),gender)]:
            score += table[(int(age),gender)][row['Model']]
        if brand != 'Luxury':
            if row['Country'] == brand:
                score += 1
        else:
            score += row['luxury_weight']
        scores.append(score)
#         ranks.append(score)
    df = df.assign(scores=pd.Series(scores).values)
    filtered = df.nlargest(int(1/2*len(df.index)), 'scores', 'first')
    return filtered


# Test
fil2 = second_stage(df, '20', 'M', 'Japan')
fil2

,Manufacturer,Model,Age,Gender,Region,Sum,EV/Hybrid available,advertisement list API,(Ryvuss Query),link to list page of Encar.com,Unnamed: 10,Car type,Price,Country,PRICE_RANK,luxury_weight,scores
1,Audi,A7,35~39,M,Seoul,958,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,BIG,5069.0,Japan,94.0,2,3
3,Audi,New A4,35~39,M,Gyeonggi-do,1144,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SMALL/COMPACT,2416.0,Japan,55.0,1,3
4,Audi,NEW A6,35~39,M,Gyeonggi-do,2253,O,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,MIDDLE,3303.0,Japan,70.0,1,3
10,BMW,3-Series (F30),35~39,M,Gyeonggi-do,2318,O,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SMALL/COMPACT,3296.0,Japan,69.0,1,3
12,BMW,4-Series (F32),30~34,M,Seoul,1124,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SPORT,2929.0,Japan,63.0,1,3
13,BMW,5 Series (G30),35~39,M,Seoul,1568,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,MIDDLE,3654.0,Japan,77.0,2,3
21,BMW,M3 (F80),35~39,M,Gyeonggi-do,493,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SPORT,4493.0,Japan,90.0,2,3
25,BMW,X5 (F15),35~39,M,Seoul,1147,O,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SUV/RV,6532.0,Japan,101.0,2,3
95,Kia,All New Canival,35~39,M,Gyeonggi-do,2572,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer...,http://www.encar.com/dc/dc_carsearchlist.do?ca...,NaN,SUV/RV,3425.0,Japan,73.0,1,3
99,Kia,all new Morning,35~39,M,Gyeonggi-do,913,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer...,http://www.encar.com/dc/dc_carsearchlist.do?ca...,NaN,SMALL/COMPACT,666.0,Japan,2.0,0,3


In [73]:
from kmodes.kmodes.kprototypes import KPrototypes

# kmodes library: https://github.com/nicodv/kmodes

def third_stage(df, wage, car_type, small_car, hybrid):
    """
    Recommends a list of cars based on specified car_type, small_car, budget1,
    budget2, using kmodes algorithm. If car_type is normal, small_car will be
    considered as a replacement. Overall, the net effect of car_type and small_car
    is one of the followings: 'MIDDLE', 'COMPACT', 'BIG', 'SUV/VR', 'SPORT', 'SMALL'

    Inputs:
    - df: data frame, to filter
    - wage: string, either '15', '15-30', '30-60', or '60'
    - car_type: string, either 'NORMAL', 'SUV/VR', or 'SPORT'
    - small_car: string, either 'SMALL/COMPACT', 'MIDDLE', or 'BIG'
    - hybrid: string, 'X' or 'O'
    """
    # According to Weiting's suggestion
    if wage == '15':
        budget = 1500/2
    elif wage == '15-30':
        budget = (4000+1500)/2
    elif wage == '30-60':
        budget = (4000+7000)/2
    else:
        budget = (16931+7000)/2
        
    X_train = df[['Price','Car type', 'EV/Hybrid available' ]].values 
    kproto = KPrototypes(n_clusters=5, init='Huang', verbose=0)
    clusters = kproto.fit(X_train, categorical=[1,2]).labels_
    if car_type != 'NORMAL':
        net_car_type = car_type
    else:
        net_car_type = small_car
    X_test = np.array([[budget, net_car_type, hybrid]])
    X_df =  pd.DataFrame(X_test)
    X_df[[0]]=X_df[[0]].astype('float')
    X_test = X_df.values
    labels = kproto.predict(X_test, categorical=[1,2])
    fil_mask = df.assign(cluster=clusters)
    fil_mask = fil_mask[fil_mask.cluster==labels[0]].drop(axis=1, columns='cluster')
    return fil_mask.nlargest(3, 'scores', 'first') # just specify the number of recommendations here
# Test
fil3 = third_stage(fil2, '30-60', 'SUV/RV', 'MIDDLE', 'O')
fil3

,Manufacturer,Model,Age,Gender,Region,Sum,EV/Hybrid available,advertisement list API,(Ryvuss Query),link to list page of Encar.com,Unnamed: 10,Car type,Price,Country,PRICE_RANK,luxury_weight,scores
25,BMW,X5 (F15),35~39,M,Seoul,1147,O,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SUV/RV,6532.0,Japan,101.0,2,3
27,BMW,X6 (F16),35~39,M,Seoul,639,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.A._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,SUV/RV,6751.0,Japan,103.0,2,2
146,Mercedes-Benz,CLS-Class W218,35~39,M,Seoul,1532,X,http://api.encar.com/search/car/list/premium?c...,(And.Hidden.N._.(C.CarType.N._.(C.Manufacturer...,http://www.encar.com/fc/fc_carsearchlist.do?ca...,NaN,MIDDLE,6865.0,Korea,104.0,2,2
